# Sentiment Analysis Part 2

_Natural Langauge Processing Nanodegree Program_

---



## Step 5: Switching gears - RNNs

We just saw how the task of sentiment analysis can be solved via a traditional machine learning approach: BoW + a nonlinear classifier. We now switch gears and use Recurrent Neural Networks, and in particular LSTMs, to perform sentiment analysis in Keras. Conveniently, Keras has a built-in [IMDb movie reviews dataset](https://keras.io/datasets/#imdb-movie-reviews-sentiment-classification) that we can use, with the same vocabulary size.

In [1]:
from keras.datasets import imdb  # import the built-in imdb dataset in Keras

# Set the vocabulary size
vocabulary_size = 5000

# Load in training and test data (note the difference in convention compared to scikit-learn)
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocabulary_size)
print("Loaded dataset with {} training samples, {} test samples".format(len(X_train), len(X_test)))

Using TensorFlow backend.


17465344/17464789 [==============================] - 0s 0us/step
Loaded dataset with 25000 training samples, 25000 test samples


In [2]:
# Inspect a sample review and its label
print("--- Review ---")
print(X_train[7])
print("--- Label ---")
print(y_train[7])

--- Review ---
[1, 175, 704, 63, 47, 6, 780, 22, 1600, 47, 49, 2, 5, 279, 49, 2140, 1740, 63, 97, 102, 15, 25, 70, 64, 340, 391, 54, 335, 6, 173, 7, 14, 704, 13, 482, 2, 9, 141, 6, 20, 10, 10, 25, 67, 4, 4588, 651, 7, 2, 926, 1641, 4014, 916, 2, 21, 36, 131, 28, 195, 278, 38, 36, 70, 4146, 2, 1880, 5, 194, 4165, 5, 36, 26, 407, 399, 6, 176, 7, 680, 183, 133, 63, 279, 186, 18, 98, 2, 88, 507, 399, 12, 143, 68, 226, 113, 39, 6, 1280, 406, 2, 25, 70, 150, 714, 794, 4, 20, 5, 30, 2, 42, 4733, 5, 106, 6, 478, 418, 7, 2, 498, 102, 10, 10, 21, 48, 25, 412, 133, 5, 25, 124, 4, 84, 25, 67, 4, 105, 11, 2, 17, 4, 2, 7, 4, 926, 6, 926, 15, 9, 170, 53, 2140, 39, 251, 8, 251, 1855, 68, 205, 2269, 15, 1337, 334, 70, 391, 4501, 4, 1031, 1510, 39, 746, 5, 25, 67, 4, 84, 1266, 11, 4, 1547, 2090, 33, 4, 1961, 890, 3130, 2, 11, 4, 172, 347, 279, 25, 909, 4403, 11, 6, 1449, 42, 6, 2, 25, 203, 1980, 279, 25, 60, 157, 19, 98, 11, 129, 292, 42, 36, 26, 581, 375, 8, 129, 313, 25, 380, 8, 784, 98, 209, 618, 127

Notice that the label is an integer (0 for negative, 1 for positive), and the review itself is stored as a sequence of integers. These are word IDs that have been preassigned to individual words. To map them back to the original words, you can use the dictionary returned by `imdb.get_word_index()`.

In [3]:
# Map word IDs back to words
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print("--- Review (with words) ---")
print([id2word.get(i, " ") for i in X_train[7]])
print("--- Label ---")
print(y_train[7])

1646592/1641221 [==============================] - 0s 0us/step
--- Review (with words) ---
['the', 'us', 'class', 'really', 'there', 'is', 'die', 'you', 'apartment', 'there', 'good', 'and', 'to', 'reason', 'good', 'relief', 'continue', 'really', 'could', 'characters', 'for', 'have', 'well', 'see', 'used', 'terrible', 'no', 'poor', 'is', 'lot', 'br', 'as', 'class', 'was', 'lead', 'and', 'it', 'should', 'is', 'on', 'i', 'i', 'have', 'can', 'of', 'mummy', 'happy', 'br', 'and', 'footage', 'food', 'snake', 'laughs', 'and', 'not', 'from', 'these', 'one', "that's", 'sense', 'her', 'from', 'well', 'glorious', 'and', 'memories', 'to', 'thought', 'satan', 'to', 'from', 'he', 'itself', 'early', 'is', 'quite', 'br', 'call', 'seems', 'scene', 'really', 'reason', 'horror', 'but', 'any', 'and', 'most', 'enjoyed', 'early', 'that', "i'm", 'were', 'script', 'acting', 'or', 'is', 'added', 'others', 'and', 'have', 'well', 'years', 'miss', 'viewers', 'of', 'on', 'to', 'at', 'and', "it's", 'accompanied', 't

Unlike our Bag-of-Words approach, where we simply summarized the counts of each word in a document, this representation essentially retains the entire sequence of words (minus punctuation, stopwords, etc.). This is critical for RNNs to function. But it also means that now the features can be of different lengths!

#### Question: Variable length reviews

What is the maximum review length (in terms of number of words) in the training set? What is the minimum?

#### Answer:

...


### TODO: Pad sequences

In order to feed this data into your RNN, all input documents must have the same length. Let's limit the maximum review length to `max_words` by truncating longer reviews and padding shorter reviews with a null value (0). You can accomplish this easily using the [`pad_sequences()`](https://keras.io/preprocessing/sequence/#pad_sequences) function in Keras. For now, set `max_words` to 500.

In [4]:
from keras.preprocessing import sequence

# Set the maximum number of words per document (for both training and testing)
max_words = 500

# TODO: Pad sequences in X_train and X_test
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

### TODO: Design an RNN model for sentiment analysis

Build your model architecture in the code cell below. We have imported some layers from Keras that you might need but feel free to use any other layers / transformations you like.

Remember that your input is a sequence of words (technically, integer word IDs) of maximum length = `max_words`, and your output is a binary sentiment label (0 or 1).

In [5]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

# TODO: Design your model
embedding_size = 64
model = Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 64)           320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 386,101
Trainable params: 386,101
Non-trainable params: 0
_________________________________________________________________
None


#### Question: Architecture and parameters

Briefly describe your neural net architecture. How many model parameters does it have that need to be trained?

#### Answer:

...

### TODO: Train and evaluate your model

Now you are ready to train your model. In Keras world, you first need to _compile_ your model by specifying the loss function and optimizer you want to use while training, as well as any evaluation metrics you'd like to measure. Specify the approprate parameters, including at least one metric `'accuracy'`.

In [6]:
# TODO: Compile your model, specifying a loss function, optimizer, and metrics
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Once compiled, you can kick off the training process. There are two important training parameters that you have to specify - **batch size** and **number of training epochs**, which together with your model architecture determine the total training time.

Training may take a while, so grab a cup of coffee, or better, go for a hike! If possible, consider using a GPU, as a single training run can take several hours on a CPU.

> **Tip**: You can split off a small portion of the training set to be used for validation during training. This will help monitor the training process and identify potential overfitting. You can supply a validation set to `model.fit()` using its `validation_data` parameter, or just specify `validation_split` - a fraction of the training data for Keras to set aside for this purpose (typically 5-10%). Validation metrics are evaluated once at the end of each epoch.

In [8]:
# TODO: Specify training parameters: batch size and number of epochs
batch_size = 64
num_epochs = 3

# TODO(optional): Reserve/specify some training data for validation (not to be used for training)
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]

# TODO: Train your model
model.fit(X_train2, y_train2,
          validation_data=(X_valid, y_valid),
          batch_size=batch_size, epochs=num_epochs)

Train on 24936 samples, validate on 64 samples
Epoch 1/2
24936/24936 [==============================] - 600s 24ms/step - loss: 0.4419 - acc: 0.7946 - val_loss: 0.4437 - val_acc: 0.8125
Epoch 2/2
24936/24936 [==============================] - 598s 24ms/step - loss: 0.3102 - acc: 0.8736 - val_loss: 0.4267 - val_acc: 0.7969


In [11]:
# Save your model, so that you can quickly load it in future (and perhaps resume training)
model_file = "rnn_model.h5"  # HDF5 file
model.save(model_file)

# Later you can load it using keras.models.load_model()
#from keras.models import load_model
#model = load_model(os.path.join(cache_dir, model_file))

Once you have trained your model, it's time to see how well it performs on unseen test data.

In [12]:
# Evaluate your model on the test set
scores = model.evaluate(X_test, y_test, verbose=0)  # returns loss and other metrics specified in model.compile()
print("Test accuracy:", scores[1])  # scores[1] should correspond to accuracy if you passed in metrics=['accuracy']

Test accuracy: 0.84876


#### Question: Comparing RNNs and Traditional Methods

How well does your RNN model perform compared to the BoW + Gradient-Boosted Decision Trees?

#### Answer:

...

## Extensions

There are several ways in which you can build upon this notebook. Each comes with its set of challenges, but can be a rewarding experience.

- The first thing is to try and improve the accuracy of your model by experimenting with different architectures, layers and parameters. How good can you get without taking prohibitively long to train? How do you prevent overfitting?

- Then, you may want to deploy your model as a mobile app or web service. What do you need to do in order to package your model for such deployment? How would you accept a new review, convert it into a form suitable for your model, and perform the actual prediction? (Note that the same environment you used during training may not be available.)

- One simplification we made in this notebook is to limit the task to binary classification. The dataset actually includes a more fine-grained review rating that is indicated in each review's filename (which is of the form `<[id]_[rating].txt>` where `[id]` is a unique identifier and `[rating]` is on a scale of 1-10; note that neutral reviews > 4 or < 7 have been excluded). How would you modify the notebook to perform regression on the review ratings? In what situations is regression more useful than classification, and vice-versa?

Whatever direction you take, make sure to share your results and learnings with your peers, through blogs, discussions and participating in online competitions. This is also a great way to become more visible to potential employers!